# Prior estimation

In [1]:
include("src/main.jl")
OPTCONFIG=OptConfig(XTOLABS=1e-8, DEBUG=true);

## The Model

In [2]:
# exponential
xlims = (0,4)
n = 200 # was 500
ndata = 100
σ = 0.3
prior = Truncated(MixtureModel([Normal(mu, .5) for mu in [1, 3]]), xlims...)


t    = Transformation(x->exp(x), x->log(x))

Transformation(#80, #81)

In [ ]:
for i = 1:100
  dir = "batch/$i"
  try
    mkdir(dir)
  end

  srand(i)
  m = FEModel(f=x->x, xs=linspace(xlims...,n), σ=σ)
  d = generatedata(m, prior, ndata);
  mt   = transformmodel(m, t)

  ncv = 20
  kfold = length(d);

  lims = ([0,4],[0,.6],[1,55], [0,.2])

  figsize=(4,3)

  ## er estimator
  regs = [ReferenceRegularizer(m, γ) for γ in linspace(0.1,1,ncv)]
  @time ss = [cvscore(m, d, d->ebprior(m, d, r), kfold) for r in regs]
  figure()
  PyPlot.plot([r.γ for r in regs], ss)
  PyPlot.savefig("$dir/ebcv.pdf")
  regR = regs[indmax(ss)].γ

  wR  = ebprior(m,  d, ReferenceRegularizer(m,  regR))
  wRt = ebprior(mt, d, ReferenceRegularizer(mt, regR))

  compareplot(m, t, pdf.(prior, m.xs), wR, wRt, "eb", lims...; figsize=figsize, savename="$dir/eb")

  ## thik estimator
  regs = [ThikonovRegularizer(γ) for γ in linspace(1,80,ncv)]
  @time ss = [cvscore(m, d, d->ebprior(m, d, r), kfold) for r in regs]
  figure()
  PyPlot.plot([r.γ for r in regs], ss)  
  PyPlot.savefig("$dir/l2cv.pdf")
  regT = regs[indmax(ss)].γ

  wT = ebprior(m,  d, ThikonovRegularizer(regT))
  wTt = ebprior(mt, d, ThikonovRegularizer(regT))

  compareplot(m, t, pdf.(prior, m.xs), wT, wTt, "L_2", lims...; figsize=figsize, savename="$dir/l2")

end

 56